Using vector semantics, we can easily convert a given text into its corresponding vector
form. Given any text, first pre- process the text and convert it into a vector using BoW
methods. Given this vector, implement your own classifier to classify the vector is predefined categories. You may use of these datasets for training and for defining the
categories:
Dataset Link: https://lionbridge.ai/datasets/14-best-text-classification-datasets-formachine-learning/

In [1]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
# from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
# from textblob import TextBlob
# from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Python program to generate word vectors using Word2Vec 

# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 

warnings.filterwarnings(action = 'ignore') 

import gensim 
from gensim.models import Word2Vec 

import pandas as pd

In [2]:
txt = pd.read_csv('/home/rupeek/Desktop/ML &AI/archive (1)/SPAM text Data.csv')
txt.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [38]:
txt.shape


(5572, 2)

In [3]:
txt.describe()


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


## Spliting the training dataset



In [40]:
#split the dataset  
#train dataset
train_reviews=txt.Category[:2572]
train_sentiments=txt.Message[:2572]
#test dataset
test_reviews=txt.Category[3000:]
test_sentiments=txt.Message[3000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(2572,) (2572,)
(2572,) (2572,)


## Text normalization



In [16]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

Removing html strips and noise text



In [18]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
txt['Category']=txt['Category'].apply(denoise_text)

Removing special characters

In [19]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
txt['Category']=txt['Category'].apply(remove_special_characters)

Removing stopwords



In [20]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
txt['Category']=txt['Category'].apply(remove_stopwords)

{'yours', 'no', 'now', 'an', 'any', 've', 'from', 'this', "mightn't", 'yourselves', "didn't", 'shouldn', 'didn', 'your', 't', 'against', 'as', 'she', 'after', 'because', 'during', 'their', 'about', "weren't", 'wasn', 'but', 'under', 'when', 'only', 'most', 'each', 'couldn', 'out', 'above', 'below', 'doesn', 'ain', 'so', 'her', 'they', 'over', 'into', "shan't", 'being', "wasn't", "that'll", 'y', 'mightn', 'a', 'of', "don't", 'to', 'can', 'haven', 'once', 'himself', 'with', 'those', 'that', 'be', 'i', 'aren', 'some', 'again', 'whom', 'he', 'by', 'our', 'you', 'should', 'then', 'are', 'who', 'had', "hadn't", "you're", 'themselves', 'it', 'hadn', 'my', 'both', 'there', 'and', 'not', 'myself', 'has', 'am', 'did', 'll', 'what', 'more', 'them', "you'd", 'have', 'his', 'its', 'is', 'where', 'own', 'we', 'm', 'been', 'down', 'if', "hasn't", 'hers', 'were', 'herself', "doesn't", 'in', 'don', 'shan', "you'll", 'hasn', 'the', "should've", 're', 'weren', "she's", "haven't", 'very', 'was', 'on', 'fu

Normalized train reviews



In [26]:
#normalized train Category
norm_train=txt.Category[:400]
norm_train[0]

'ham'

Normalized test reviews



In [27]:
norm_test=txt.Category[300:]
norm_test[305]

'spam'

Bags of words model

It is used to convert text documents to numerical vectors or bag of words.



In [35]:
from sklearn.feature_extraction.text import CountVectorizer
# sentences = ['We are using the Bag of Word model.',
#              'Bag of Word model is used for extracting the features.',
#             "We live in India. india india"]
cv = CountVectorizer()
BOW = cv.fit_transform(txt)
print(BOW)
print(cv.vocabulary_)
cv.get_feature_names()

  (0, 0)	1
  (1, 1)	1
{'category': 0, 'message': 1}


['category', 'message']

In [45]:
from sklearn.preprocessing import LabelBinarizer

#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(txt['Category'])
print(sentiment_data.shape)


(5572, 1)


In [50]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:3572]
test_sentiments=sentiment_data[2000:]
print(train_sentiments.shape)
print(test_sentiments.shape)

(3572, 1)
(3572, 1)


In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_sentiments,test_sentiments, test_size=0.2, random_state=0)

In [52]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators=800, random_state=100)
model.fit(X_train,y_train)
test_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, test_pred)

0.8587412587412587